$\newcommand{\R}{\mathbb{R}}$
$\newcommand{\N}{\mathbb{N}}$
$\newcommand{\transp}{\mathrm{T}}$

# A. Linear-nonlinear impulse systems

A major drawback of hybridization techniques is the explosion in the number of cells. In the following we explore an alternative or dual approach, which is more like a *global linearization*.

In the following, the guiding principle is to *embed* the non-linearities in a higher-dimensional space, and exploit the power of support-function methods, namely those very optimized in SpaceEx. Ok, but where do the non-linearities go? Actually with Carleman linearization we find out that the system can be described by linear evolutions alternated with nonlinear impulses. 

The notion is linear impulsive system is well-known in control theory, and has to do with dynamics that are linear and receive linear impulses at some collection of time events. We shall generalize to linear-nonlinear systems, which are linear systems but suffer kicks which are polynomial. 

The motivation is that the cost associated to the computation is packed onto these switches, since the linear part is very efficient. One is left to the wishful thinking that these events are relatively small. 

The method of Carleman linearization has a long history, first mentioned by T. Carleman in relation with linearization methods in the theory of integral equations. We remark that [is a new feature of Mathematica 10](https://www.wolfram.com/mathematica/new-in-10/nonlinear-control-systems/carleman-linearization.html). 

Our contribution [ongoing with G. Frehse and A. Pouly] is to show that Carleman linearization can be used for validated integration. We have in the linear regimes that

$$
x(t) \in \hat{x}\oplus e(t),
$$

for $e(t)$ some known function. Moreover, there is only one matrix $A$ that has to be computed.


## 1. Kronecker product 

**Motivation**. What is a handy notation for all possible products of elements of a vector?

### 1.1 Kronecker product of vectors

For any pairs of vectors $x \in \R^n$, $y\in \R^m$, their *Kronecker product* is
$$
z = x\otimes y = (x_1 y_1, x_1 y_2,\ldots, x_1 y_m, x_2 y_1,\ldots,x_2 y_m, \ldots, x_n y_1,\ldots, x_n y_m)^\transp.
$$
Note that it is not commutative, $x \otimes y \neq y \otimes x$, in general. Given $x \in \R^n$, its $i$-th *Kronecker power* is defined as
$$
 x^{[i]} =  \underset{\text{i times}}{\underbrace{x\otimes \ldots \otimes x}} = \overset{i}{\otimes} x. 
$$
Note that $\dim x^{[i]} = n^i$. 

 For instance, let $x \in \R^n$. We want $x^{[i]}$ to denote all possible products of elements of $x$ whose powers sum exactly $i$. For instance, if $n=2$, 
$$
 x^{[1]}= x,\qquad x^{[2]} = (x_1^2,x_1 x_2,x_2 x_1,x_2^2)^\transp, 
$$
and
$$
 x^{[3]} = (x_1^3,x_1^2 x_2, x_1^2 x_2, x_1 x_2^2,    x_2 x_1^2,x_2^2 x_1 , x_1 x_2^2,x_2^3 )^\transp.
$$

In [10]:
%display typeset

def kron_prod(x,y):
    z = [x[i]*y[j]  for i in range(len(x)) for j in range(len(y))]
    return z

x = [var('x'+str(i+1)) for i in range(2)]; y = [var('y'+str(i+1)) for i in range(4)]
kron_prod(x,y)

[x1*y1, x1*y2, x1*y3, x1*y4, x2*y1, x2*y2, x2*y3, x2*y4]

### 1.2. Kronecker product of matrices 

For matrices the definition is analogous: let $A \in \R^{n\times m}$ and $B \in \R^{p \times q}$. Then $C = A\otimes B \in \R^{np \times mq}$, and is given by
$$
C = A \otimes B = \begin{pmatrix}
a_{11}B &\ldots&a_{1m}B \\
\vdots & & \vdots \\
a_{n1}B & \ldots &a_{nm}B 
\end{pmatrix}.
$$ 

In [9]:
# in sagemath we can use A.tensor_product(B) 
In = identity_matrix(2) 
In.tensor_product(In)

[1 0|0 0]
[0 1|0 0]
[---+---]
[0 0|1 0]
[0 0|0 1]

### 1.3 Norm properties

Hereafter $||\cdot||$ denote the supremum norm, $||x|| = \sup\limits_{i=1,\ldots,n} |x_i|$. The following formulas can be easily proved and are often used:

1. $||x\otimes y|| = ||x|| ~||y||$.
2. $||A\otimes B|| = ||A||~||B||$, where $||A|| = \sup_{||x|| =1} ||Ax||$ for any $A = (a_{ij}) \in \R^{n\times n}$.

# 2. Carleman linearization


Consider the initial value problem 

$$
\dfrac{dx}{dt} = F_1 x + F_2 x^{[2]} + \ldots +  F_k x^{[k]},\qquad x(0) = x_0 \in \R^n. \label{eq:S_ode}
$$

** Definition **. For any $i,j \in \N$ we define $A^i_{i+j-1} \in \R^{n^i \times n^{i+j-1}}$ as being
\begin{align}
 A^i_{i+j-1} &:= \underset{\text{i factors}}{\underbrace{F_j \otimes I_n \otimes \ldots  \otimes I_n}} + I_n \otimes F_j \otimes  \ldots  \otimes I_n + \ldots +  I_n \otimes \ldots  \otimes I_n \otimes F_j \nonumber \\ 
 &= \sum_{\nu=1}^i (  I_n \otimes \ldots \otimes \underset{\underset{\nu-\text{th position}}{\uparrow}}{F_j} \otimes  \ldots  \otimes I_n ). \label{eq:defiBij}
\end{align}


The $i$-th block of auxiliary variables, $y_i \in \R^{n^i}$, is defined as the Kronecker power 
$$
y_i := x^{[i]},\qquad i\in \N. 
$$



## 2.1. Infinite-dimensional embedding

** Proposition **. Let $x : I \to \R^n$ be a solution, and $y_i = x^{[i]}$ for $i \in \N$. Then,
$$
\dfrac{d y_i}{dt} = \sum_{j=0}^{k-1} A^i_{i+j} y_{i+j},\qquad i \in \N.  
$$

** Proposition **. For all $i\geq 1$, $0\leq j \leq k-1$, the estimate $||A^i_{i+j}|| \leq i ||F_{j+1}||$ holds.

In matrix form, we concatenate all blocks $y_i$ and define the infinite-dimensional vector $y \in \R^\N$, that is, $y := (y_1,y_2,\ldots)^\transp$. Then, $y(t)$ satisfies the initial value problem 
\begin{equation}
\dfrac{dy}{dt}=Ay,\qquad y_i(0) = (x(0))^{[i]} = x_0^{[i]}~~ \forall i \in \N, \label{eq:InfSysMatrix}
\end{equation}
where $A$ is the infinite-dimensional block upper-triangular matrix,
\begin{equation}
A = \begin{pmatrix}
  A_1^1 & A_2^1 & \ldots & A_{k}^1 & & & \\ 
   & A_2^2 & A_3^2 & \ldots & A_{k+1}^2 & & & \\
   &  & \ddots  &  &   & \ddots & \\   
 &  &   & A_N^{N} &  A_{N+1}^{N} & \ldots & A_{N+k-1}^N  \\   
&  &   &  &  \ddots &  & \ddots \\ 
 \end{pmatrix}. 
\end{equation}

## 2.2. Truncation

Now we move on to consider the truncated system of $N$ variables. Here we choose the \emph{null closure conditions}, that is, we just eliminate the dependence of those whose neighbours exceed $N$. Notice that we have to classes of variables: those which are not influenced by the truncation and those which are. Since each site $i$ is influenced by its right-neighbour far at $k-1$ positions at most, then those influenced by the truncation start at site $N-k+2$.
$$
\dfrac{d \hat{y}_i}{dt} = \sum\limits_{j=0}^{k-1} A^i_{i+j} \hat{y}_{i+j}, \qquad i = 1,\ldots,N-k+1
$$
and 
$$
\dfrac{d \hat{y}_i}{dt} = \sum\limits_{j=0}^{\ell-1}  A^i_{i+j} \hat{y}_{i+j}, \qquad i = N-k+2,\ldots,N
$$
where $\ell=\min\{k,N+1-i\}$. Equivalently, we can express this system in matrix form, if we consider the vector $\hat{y} = (\hat{y}_1, \hat{y}_2, \ldots, \hat{y}_N)^\transp$. Then $\hat{y}$ satisfies the linear system
$$
\dfrac{d \hat{y}}{dt} = B_N \hat{y},\qquad \hat{y}_i(t_0) = x^{[i]}(0) = x_0^{[i]}  ~~ \forall i = 1,\ldots,N
$$
Here, $B_N$ is the finite-dimensional block upper-triangular matrix,
$$
B_N = \begin{pmatrix}
  A_1^1 & A_2^1 & \cdots & A_k^1 & &  \\ 
   & A_2^2 & A_3^2 & \cdots & A_{k+1}^2 & &  \\
   &  &  & \ddots  &     & \\   
 &  &   &  &   A_{N-1}^{N-1} & A_{N}^{N-1}  \\   
&  &   &  &     & A_{N}^N \\ 
 \end{pmatrix}. 
$$
Note that $\dim A^i_{i+j-1} = n^i \times n^{i+j-1}$ for $i=1,\ldots,N$ and $j=1,\ldots,k$. Thus, $B_N$ is a square matrix of order $\frac{n^{N+1}-n}{n-1}$.


*Exercise*. Compute the Carleman linearization of a scalar quadratic system $\dot{x}=ax+bx^2$. 

*Remark*. In this case it is possible to solve in closed form the infinite-dimensional system. One can see that it present smooth non-analytic solutions. However, it can be proved that the space of solutions of the infinite-dimensional system restricted to analytic functions coincides with that of the original system.